In [ ]:
%pip install langsmith

In [ ]:
from langsmith import Client
# 测试连接
try:
    client = Client()
    for p in  client.list_projects():
        print(p)
    # print("连接成功！项目列表:", client.list_projects())
except Exception as e:
    print("连接失败:", str(e))

In [ ]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env",override=True)
from os import getenv

import os

key = getenv("SF_API_KEY") 
url = getenv("SF_API_BASE")
model = getenv("SF_MODEL")

os.environ["DEEPSEEK_API_KEY"] = key
os.environ["DEEPSEEK_API_BASE"] = url
print(key, url, model)

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel


llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)

# 定义子链
prompt = ChatPromptTemplate.from_template("总结文本：{text}")
# debug_prompt = prompt | (lambda x: print("提示内容:\n", x.to_messages()) or x)
summary_chain = (
    prompt #| (lambda x: print("提示内容:\n", x.to_messages()) or x)
    | llm 
    | StrOutputParser()
)

sentiment_chain = (
    ChatPromptTemplate.from_template("分析文本情感倾向：{text}")
    | llm 
    | StrOutputParser()
)

# 并行处理链
parallel_chain = RunnableParallel({
    "summary": summary_chain,
    "sentiment": sentiment_chain
})

# 执行
input_data = {"text": "这款手机拍照效果出色，但电池续航较差"}
result = parallel_chain.invoke(input_data)